## Assignment 1: HTML & Markdown

Create a webpage mimicking the layout of the page in results preview. Don't worry about perfection, the main goal is to get practice with the basic html and markdown components we covered. 

* I've provided the style for the outer Div. The main focus here is the components rather than style.

* If you wish, look up your own style arguments to further modify the page.

* The style argument for the second Markdown list should be: `style={"color":"limegreen"}`.

* To make Line Breaks, use html.Br() in between other html components.

In [6]:
app = JupyterDash(__name__)

app.layout = html.Div(
    style={
        "font-family":"Arial", 
        "backgroundColor": "black", 
        "color": "lightGrey",
    },
)


app.run_server(debug=True, mode="inline", port=8649)

## Assignment 2: App Styling

Take some time to style the map application we build at the end of Section 3. I've pasted my code below.

* Style the background, font color and font family of the application. I've made mine `darkblue`, `lightgrey`, and `arial`, but feel free to apply your own styles. 
* Modify the font sizes of the slider and check list components.
* Feel free to make any other tweaks to your application! To modify the area outside of the maps (mine is `lightblue`), we modify the `geo_bgcolor` attribute.

In [ ]:
from dash import dcc, html
from jupyter_dash import JupyterDash

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("../data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2(id="title"),
    html.P("Select Options Below:"),
    dcc.Slider(
        id="Elevation Slider",
        min=0,
        max=4000,
        step=500,
        value=500,
    ),
    dcc.Checklist(
        id="Feature Checklist",
        options=["Has Snow Park", "Has Night Ski"], 
        value=["Has Snow Park", "Has Night Ski"], 
    ),
    dcc.Graph(id="graph")
])


@app.callback(
    Output('title', 'children'),
    Output('graph', 'figure'),
    Input("Elevation Slider", "value"),
    Input("Feature Checklist", "value"),
)

def plot_lift_number(elevation, features):

    ski_resorts_filtered = ski_resorts.query("HighestPoint > @elevation")
    
    title = f"Ski Resorts with Elevation Over {elevation}M Max Elevation"
    
    if features == []:
        df = (ski_resorts_filtered
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    elif len(features)==2:
        df = (ski_resorts_filtered
              .query("Snowparks == 'Yes' and NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    elif features == ["Has Snow Park"]:
        df = (ski_resorts_filtered
              .query("Snowparks == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    else:
        df = (ski_resorts_filtered
              .query("NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    fig = px.choropleth(
        df,
        locations="Country",
        color="ResortCount",
        locationmode="country names",
        scope="europe"
    ).update_geos(fitbounds="locations").update_layout(
        margin={"r":0,"t":0,"l":0,"b":0}, 
        coloraxis_colorbar_x=.85,
#         width=1000,
#         height=600
    )

    return title, fig 
    
if __name__ == "__main__":
    app.run_server(port=2038)

## Assignment 3: Dashboard Layouts

Modify the layout of the application from Assignment 2.

You are really free to do as you wish, but make sure to try: 
* Applying a dbc theme of your choice to all components.
* Using a grid based layout to organize your application.
* Feel free to make any other tweaks to your application! We will get more practice in the project, but feel free to try adding a tab based structure here even if the second tab is empty.